In [40]:
!pip install fastparquet
!gdown --id '1cAthveg1d3MjrKJtMKGzfX3eH8HJ-dQp'
!gdown --id '1-RxLiAq8SBsKZECM7vno8eSazvZ-jMbG'
!unzip MedNLI_dataset.zip
!pip install stanza
!pip install medialpy
!pip install contractions
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.wv.vectors.npy
!wget https://huggingface.co/garyw/clinical-embeddings-100d-w2v-cr/resolve/main/w2v_OA_CR_100d.bin.trainables.syn1neg.npy

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1cAthveg1d3MjrKJtMKGzfX3eH8HJ-dQp
To: /content/MedNLI_dataset.zip
100% 681k/681k [00:00<00:00, 8.65MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1-RxLiAq8SBsKZECM7vno8eSazvZ-jMbG
From (redirected): https://drive.google.com/uc?id=1-RxLiAq8SBsKZECM7vno8eSazvZ-jMbG&confirm=t&uuid=d9da6ffd-b989-4ed7-9350-4278a1e601cd
To: /content/modeln.pth
100% 515M/515M [00:06<00:00, 77.1MB/s]
Archive:  MedNLI_dataset.zip
replace MedNLI_dataset/valid-00000-of-00001-cc552de6d1a6fa4b.parquet? [y]es,

In [1]:
# @title imports
from transformers import AutoTokenizer, AutoModel
import pandas as pd
import stanza
import shutil
import numpy as np
from numpy.linalg import norm
import string
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gensim.models import FastText, Word2Vec, KeyedVectors
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
import medialpy
from stanza import Pipeline
import contractions

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# @title CODES
def find_pre_and_hyp(query):
    start_pre = query.find("[PRE]") + len("[PRE]")
    end_pre = query.find("[HYP]")
    start_hyp = query.find("[HYP]") + len("[HYP]")
    end_hyp = query.find("OUTPUT:")
    premise = query[start_pre:end_pre].strip()
    hypothesis = query[start_hyp:end_hyp].strip()

    return premise,hypothesis


class CustomDataset(Dataset):
    def __init__(self, x_list, y_list):
        self.samples = []
        for x,y in zip(x_list,y_list):
            #x_tensor = torch.tensor(x,dtype = torch.float32)
            y_tensor = torch.tensor(y,dtype = torch.float32)
            self.samples.append((x[0],x[1],y_tensor))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]


def get_lists(data):
    x_list = []
    y_list = []
    i =0
    for query,answer in zip(data['query'],data['answer']):
        i = i + 1
        if answer == 'entailment':
           y = [1,0,0]
        elif answer == 'neutral':
           y = [0,1,0]
        elif answer == 'contradiction':
           y = [0,0,1]
        else:
           print('should not get here')

        premise,hypothesis = find_pre_and_hyp(query)
        x_list.append((premise,hypothesis))
        y_list.append(y)
    return x_list,y_list


class embedding_layer(nn.Module):
  def __init__(self,bert_model,tokenizer):
    super(embedding_layer, self).__init__()
    self.bert_model = bert_model
    self.tokenizer = tokenizer

  def forward(self, x):
    with torch.no_grad():
         s = tokenizer(x,return_tensors="pt",padding=True).to(device)
         vec = bert_model(**s)['last_hidden_state'].to(device)
    return vec

class PositionalEncoding(nn.Module):
    def __init__(self, seq_len,embedding_size):
        super().__init__()
        self.dropout = nn.Dropout(0.1)
        self.embedding_size = embedding_size
        self.seq_len = seq_len

    def forward(self, x):
        pe = torch.zeros(x.size(0), x.size(1), self.embedding_size).to(device)
        div_term = torch.zeros(x.size(0), 1, self.embedding_size).to(device)
        ks = torch.arange(self.embedding_size).float().to(device)
        values = torch.exp(-torch.log(torch.tensor(1000.0)) * 2 * ks / self.embedding_size).to(device)
        values = values.view(1, 1, -1).to(device)
        div_term = div_term + values
        x = x.reshape([x.shape[0],x.shape[1],1]).to(device)
        pe[:, :, ::2] = torch.sin(x * div_term)[:, :, ::2].to(device)
        pe[:, :, 1::2] = torch.cos(x * div_term)[:, :, 1::2].to(device)
        return self.dropout(pe)

class MultiHeadAttention(nn.Module):
    def __init__(self, embedding_size, heads):
        super().__init__()
        self.embedding_size = embedding_size
        self.heads = heads
        self.head_dim = embedding_size // heads
        assert(self.heads * self.head_dim == self.embedding_size), "Invalid number of heads"
        self.fc_values = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_keys = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_queries = nn.Linear(self.head_dim, self.head_dim, bias=False).to(device)
        self.fc_out = nn.Linear(heads * self.head_dim, embedding_size).to(device)

    def forward(self, values, keys, queries, mask):
        N = queries.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], queries.shape[1]
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = queries.reshape(N, query_len, self.heads, self.head_dim)
        values = self.fc_values(values).to(device)
        keys = self.fc_keys(keys).to(device)
        queries = self.fc_queries(queries).to(device)
        energy = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]).to(device)
        if mask is not None:
            energy = energy.masked_fill(mask.to(device) == 0, float("-1e20")).to(device)
        energy = torch.softmax(energy / (self.embedding_size ** 0.5), dim=3).to(device)
        attention = torch.einsum("nhql,nlhd->nqhd", [energy, values]).to(device)
        attention = attention.reshape(N, query_len, self.heads * self.head_dim).to(device)
        out = self.fc_out(attention)
        return out

class TransformerBlock(nn.Module):
    def __init__(self, embedding_size, heads, forward_expansion, p):
        super().__init__()
        self.attention = MultiHeadAttention(embedding_size, heads)
        self.norm1 = nn.LayerNorm(embedding_size)
        self.feed_forward = nn.Sequential(nn.Linear(embedding_size, forward_expansion * embedding_size),
                                          nn.ReLU(),
                                          nn.Linear(forward_expansion * embedding_size, embedding_size))
        self.norm2 = nn.LayerNorm(embedding_size)
        self.dropout = nn.Dropout(p)
    def forward(self, values, keys, queries, mask):
        attention_out = self.attention(values, keys, queries, mask)
        x = self.norm1(attention_out + queries)
        x = self.dropout(x)
        ff_out = self.feed_forward(x)
        out = self.norm2(ff_out + x)
        out = self.dropout(out)
        return out

class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_size, num_layers, heads,
                 forward_expansion, max_length, p, device):
        super().__init__()
        self.device = device
        self.word_embedding = embedding_layer(bert_model,tokenizer)
        self.positional_embedding = PositionalEncoding(max_length, embedding_size)
        self.layers = nn.ModuleList([TransformerBlock(embedding_size, heads, forward_expansion, p) for _ in range(num_layers)])
        self.dropout = nn.Dropout(p)

    def forward(self, x, mask):
        mask = None
        pe = self.word_embedding(x)
        N = pe.size(0)
        seq_len = pe.size(1)
        positions = torch.arange(0, seq_len).expand(N, seq_len).to(self.device)
        out = self.dropout((pe + self.positional_embedding(positions)))
        for layer in self.layers:
            out = layer(out, out, out ,mask)
        return out


class Transformer(nn.Module):
    def __init__(self, src_vocab_size, src_pad_idx, embedding_size=768,
                 num_layers=1, forward_expansion=8, heads=8, max_length=100, p=0.1):
        super().__init__()
        self.src_pad_idx = src_pad_idx
        self.encoder = Encoder(src_vocab_size, embedding_size, num_layers, heads,
                               forward_expansion, max_length, p, device)

    def get_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2).to(device)
        return src_mask

    def forward(self, src):
        src_mask = None
        enc_out = self.encoder(src, src_mask).to(device)
        return enc_out

def get_model_acc(model,data_loader):
    model.eval()
    predictions = []
    true_labels = []
    for x_pre, x_hyp, y in data_loader:
        outputs = model(x_pre,x_hyp)
        predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(predicted_labels)
        true_label = torch.argmax(y, dim=1).cpu().numpy()
        true_labels.extend(true_label)
    model.train()
    return accuracy_score(true_labels, predictions)


def get_inf_from_num(num):
    if num == 0:
       return 'entailment'
    elif num == 1:
       return 'neutral'
    elif num == 2:
       return 'contradiction'

class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.1)
        self.transformer1 = Transformer(src_vocab_size, src_pad_idx).to(device)
        self.transformer2 = Transformer(src_vocab_size, src_pad_idx).to(device)
        self.fc1 = nn.Linear(768, 256).to(device)
        self.fc2 = nn.Linear(256, 3).to(device)

    def deabbreviation(self,text):
        try:
           return medialpy.find(text).meaning[0]
        except:
           return text

    def preprocess_text(self,text):
        text = contractions.fix(text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text)
        ntokens = []
        for token in tokens:
            ntokens.append(self.deabbreviation(token).lower())
        stop_words = set(stopwords.words('english'))
        stop_words.remove('no')
        stop_words.remove('not')
        filtered_tokens = [word for word in ntokens if word not in stop_words]
        processed_text = ' '.join(filtered_tokens)
        return processed_text

    def forward(self, x_pre,x_hyp):
        nx_pre = ()
        nx_hyp = ()
        for t in x_pre:
            nx_pre += (self.preprocess_text(t),)
        for t in x_hyp:
            nx_hyp += (self.preprocess_text(t),)
        x_pre = nx_pre
        x_hyp = nx_hyp
        enc_x_pre = self.transformer1(x_pre)
        enc_x_hyp = self.transformer2(x_hyp)
        enc_x_pre = torch.mean(enc_x_pre,1)
        enc_x_hyp = torch.mean(enc_x_hyp,1)
        #x = torch.cat((enc_x_pre,enc_x_hyp),1)
        x = enc_x_pre * enc_x_hyp
        x = F.relu(self.fc1(self.dropout1(x)))
        x = nn.functional.softmax(self.fc2(self.dropout2(x)),dim=1)
        return x

def get_inf_from_num(num):
    if num == 0:
       return 'entailment'
    elif num == 1:
       return 'neutral'
    elif num == 2:
       return 'contradiction'

In [3]:
test_data = pd.read_parquet('MedNLI_dataset/test-00000-of-00001-47685aa42db61e77.parquet', engine='fastparquet')
train_data = pd.read_parquet('MedNLI_dataset/train-00000-of-00001-210cfe9263b99806.parquet', engine='fastparquet')
valid_data = pd.read_parquet('MedNLI_dataset/valid-00000-of-00001-cc552de6d1a6fa4b.parquet', engine='fastparquet')

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("gsarti/biobert-nli")
bert_model = AutoModel.from_pretrained("gsarti/biobert-nli").to(device)
src_pad_idx = 0
src_vocab_size = bert_model.config.vocab_size
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma,depparse')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json


INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [10]:
train_x_list,train_y_list = get_lists(train_data)
test_x_list,test_y_list = get_lists(test_data)
val_x_list,val_y_list = get_lists(valid_data)

In [11]:
train_dataset = CustomDataset(train_x_list,train_y_list)
test_dataset = CustomDataset(test_x_list,test_y_list)
val_dataset = CustomDataset(val_x_list,val_y_list)

In [12]:
bsize = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bsize, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=bsize, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
with torch.no_grad():
     model = torch.load('modeln.pth')
     print(get_model_acc(model,test_loader))

0.7538677918424754


In [14]:
def get_wordsrel(text):
    sent = nlp(text).sentences[0]
    words_rel = []
    wordsandheads = []
    for word in sent.words:
        wordsandheads.append((word.deprel,word.lemma,word.head))

    for word in sent.words:
        for wordandhead in wordsandheads:
            if word.id == wordandhead[2]:
               words_rel.append((wordandhead[0],word.lemma,wordandhead[1]))
    for wordandhead in wordsandheads:
            if wordandhead[2] == 0:
               words_rel.append((wordandhead[0],None,wordandhead[1]))
    return words_rel

In [57]:
def deabbreviation(text):
    try:
        return medialpy.find(text).meaning[0]
    except:
        return text

def preprocess_text(text):
    text = contractions.fix(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    ntokens = []
    for token in tokens:
        ntokens.append(deabbreviation(token).lower())
    stop_words = set(stopwords.words('english'))
    stop_words.remove('no')
    stop_words.remove('not')
    stop_words.remove('of')
    filtered_tokens = [word for word in ntokens if word not in stop_words]
    processed_text = ' '.join(filtered_tokens)
    return processed_text

In [58]:
def Rule1(xpre,xhyp):
    prerel = get_wordsrel(preprocess_text(xpre))
    hyprel = get_wordsrel(preprocess_text(xhyp))
    return  len(prerel)<2 and len(hyprel)<5

In [59]:
def get_hybrid_answer(xpre,xhyp,dp_answer):
    hy_answer = dp_answer
    if Rule1(xpre,xhyp):
       hy_answer = 'neutral'
    return hy_answer

In [62]:
def X(model,data_loader):
    model.eval()
    predictions = []
    true_labels = []
    xpre = []
    xhyp = []
    for x_pres, x_hyps, y in data_loader:
        for i in range(len(x_pres)):
            xpre.append(x_pres[i])
            xhyp.append(x_hyps[i])
        outputs = model(x_pres,x_hyps)
        predicted_labels = torch.argmax(outputs, dim=1).cpu().numpy()
        predictions.extend(predicted_labels)
        true_label = torch.argmax(y, dim=1).cpu().numpy()
        true_labels.extend(true_label)
    w = 0
    for i in range(len(xpre)):
        dp_answer = get_inf_from_num(predictions[i])
        hy_answer = get_hybrid_answer(xpre[i],xhyp[i],dp_answer)
        ground_truth = get_inf_from_num(true_labels[i])
        if hy_answer != ground_truth:
        #if Rule1(xpre[i],xhyp[i]):
           w += 1
           print('num: '+str(i))
           print('w: '+str(w))
           print('premise: '+xpre[i])
           print('hypothesis: '+xhyp[i])
           print('dp_answer:'+dp_answer)
           print('ground_truth:'+ground_truth)
           print('=====================================')
    model.train()

In [63]:
with torch.no_grad():
     model = torch.load('modeln.pth')
     X(model,test_loader)

num: 5
w: 1
premise: 63 year old woman with known history of hypercholesterolemia and peptic ulcer disease s/p gastric bypass some years ago was involved in a low-speed MVC.
hypothesis: the patient has no significant injuries
dp_answer:contradiction
ground_truth:neutral
num: 6
w: 2
premise: HISTORY OF PRESENT ILLNESS:  The patient is a 54 year old male with endstage renal disease secondary to type 1 diabetes who presents for kidney transplant from wife.
hypothesis: patient is on insulin
dp_answer:neutral
ground_truth:entailment
num: 11
w: 3
premise: HISTORY:  [**Known patient lastname **] is a 33-6/7 weeks male, twin #2, born at 0307 p.m. on [**2749-6-23**] via C-section for preeclampsia to a 34- year-old, G1, para 0, now 2, mother with an [**Name (NI) 2016**] of [**2749-8-5**].
hypothesis: the patient has low birth weight
dp_answer:entailment
ground_truth:neutral
num: 14
w: 4
premise: He denied headache or nausea or vomiting.
hypothesis: He is afebrile
dp_answer:entailment
ground_trut